In [1]:
from bs4 import BeautifulSoup as BS
import urllib.request as req
import urllib.parse as pa
import pandas as pd
from tqdm import tqdm_notebook
import time
import re
import math

In [2]:
# 평점 url 얻기 위한 url_open

def url_point_Open():
    url = 'https://movie.naver.com/movie/bi/mi/point.nhn?code=186615'
    url_open = req.urlopen(url)
    
    return url_open

In [3]:
def url_Open(sub_url, page_num):
    url = 'https://movie.naver.com' + sub_url +'&page='+ str(page_num)
    url_open = req.urlopen(url)
    
    return url_open

In [4]:
soup = BS(url_point_Open(), 'html.parser')
ifr_url = soup.find('iframe')['src']
ifr_url

'/movie/bi/mi/pointWriteFormList.nhn?code=186615&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

In [5]:
soup = BS(url_Open(ifr_url, 11), 'html.parser')

In [6]:
count = soup.find('strong', 'total').find('em')
count

<em>4,813</em>

In [7]:
count = int(count.string.replace(',', ''))

In [8]:
count

4813

In [9]:
loop_count = math.ceil(count/10)
loop_count

482

In [20]:
point = []
review = []
date = []

In [21]:
page_num = 1

while page_num  < loop_count+1:
    soup = BS(url_Open(ifr_url, page_num), 'html.parser')
    result = soup.find(class_='score_result')
    lis = result.find_all('li')
    
    for lst in lis:
        count_span = len(lst.find(class_='score_reple').find_all('span'))
        count_icon = len(lst.find_all(class_='ico_viewer'))
        count_spo = len(lst.find_all(class_='text_spo'))
        point.append(int(lst.find(class_='star_score').find('em').get_text()))
#         print(count_span, count_icon, count_spo)
        
        if count_icon ==1 and count_span == 2:
            r = lst.find(class_='score_reple').find('p').get_text().strip().replace('\n', '').replace('\r','').replace('\t', '').replace('관람객', '')
        elif count_span == 2:
            r = lst.find(class_='score_reple').find('p').get_text().strip()
        elif count_icon ==1 and count_spo == 1:
            r = lst.find_all('span')[4].get_text().strip()
        elif count_icon == 1:
            r = lst.find(class_='score_reple').find('p').get_text().strip().replace('\n', '').replace('\r', '').replace('\t', '').replace('관람객', '')
        elif count_spo == 1:
            r = lst.find_all('span')[3].get_text().strip()
        else:
            r = lst.find_all('a')[0]['data-src']
        review.append(r)
        date.append(lst.find(class_='score_reple').find_all('em')[1].get_text())
            
    page_num += 1
    
    

    

In [22]:
df = pd.DataFrame(dict(zip(['날짜','리뷰', '평점'], [date, review, point])))
df

,날짜,리뷰,평점
0,2019.12.31 09:48,미드웨이가 재밌으면 추천 백두산이 재밌으면 비추,9
1,2019.12.31 10:41,저 해전이 있었기에 우리나라 광복도 가능 했음,10
2,2019.12.31 09:38,백두산 상영관 대폭줄이고 미드웨이 상영관 대폭늘려라,10
3,2019.12.31 09:14,방금 개봉했는데 1점 준애는 뭐냐 ㅋㅋ 일본놈이냐? 이제 광고돌고 있을 시간이다.,10
4,2019.12.31 19:58,교과서에는 미드웨이 해전에서 미국이 승리했다고 한 줄로 서술되어 있지만 단순히 한 ...,10
...,...,...,...
4808,2019.12.31 09:11,대만에서 10월31일 개봉했었는데.성공못했음. 미국에서도 성적이 별로구..그이유가 ...,1
4809,2020.01.05 20:41,아니 이게 재밌다는 사람은 뭐야?;그냥 국뽕->미국뽕 이 차이밖에 없는데 이 영화가...,2
4810,2020.01.01 23:40,시간이 아까워요. 엉성한 그래서 감동도 교훈도 없는...ㅉㅉ,2
4811,2019.12.31 15:37,"진주만보다 낫다고해서봤는데대실망...천조국,왜놈들 국뽕영화",4


In [15]:
df.to_csv('movie.csv', encoding='utf-8')

In [16]:
# 이게 이렇게 작동하면 안되는데 작동이 된다... 허허